# rosalie dev

Notebook purpose:

- Develop `rosalie` package and templates for working with it

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

from causaljet.experiment_evaluation.models import Cuped

import rosalie as ro
import rosalie.utils.helpers as hp

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)

%config InlineBackend.figure_format ='retina'
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [2]:
# Load data

UNIT_LEVEL = "customer"
UNIT_ID = 'id'
METRIC = 'order_price'
PRE_PERIOD = '1 Jan 2023', '31 May 2023'
POST_PERIOD ='1 Jun 2023', ' 31 Aug 2023'
FORMAT = 'cross_section'

df = (
    hp.read_data(UNIT_LEVEL)
    .pipe(hp.add_pre_metric_value, METRIC, PRE_PERIOD, POST_PERIOD)
)
hp.data_info(df)

Reading data from cache...
Shape: (62279, 3)
Units: 62,279


,id,order_price,order_price_pre
53943,JE:IE:1000071,22.625000,27.049999
53944,JE:IE:1000153,14.245000,17.390625
53945,JE:IE:100021,12.950000,30.600000
53947,JE:IE:1000294,23.299999,29.455999
53948,JE:IE:1000373,27.950001,26.139999


In [16]:

preprocessors = []
evaluators = []


def preprocessor(func):
    preprocessors.append(func)
    return func


def evaluator(func):
    evaluators.append(func)
    return func


# @preprocessor -- do before simulation starts
def add_pre_metric_value(df):
    """Add pre-period metric value for CUPED adjustment."""
    metric = "order_price"
    pre_period = '1 Jan 2023', '31 May 2023'
    post_period ='1 Jun 2023', ' 31 Aug 2023'
    post = (
        df.loc[slice(*post_period)]
        .groupby("user_id")
        [metric].mean()
        .astype('float32')
        .rename(f"{metric}")
        .reset_index()
    )
    pre = (
        df.loc[slice(*pre_period)]
        .groupby("user_id")
        [metric].mean()
        .astype('float32')
        .rename(f"{metric}_pre")
        .reset_index()
    )
    result = pd.merge(post, pre, how="left").dropna()
    return result


# @evaluator
def wls(df, metric):
    y = df[metric]
    x = sm.add_constant(df["is_treated"].astype(float))
    w = df["assignments_freq"]
    model = sm.WLS(endog=y, exog=x, weights=w)
    results = model.fit()
    return results.pvalues["is_treated"]


@evaluator
def causal_jet_cuped(df, metric):
    """Run Causal Jet CUPED implementation and return p-value.

    Because data is already pre-processed, we only need to supply the following:
    - A cross-section dataframe with `metric` and `metric_pre` columns to `ass_w_cov_panel_df`
    - The metric name to `metric_name`
    - The unit identifier to `unit_identifier`

    All other parameters can be left as default.
    """
    result = Cuped(
        ass_w_cov_panel_df=df,
        metric_name=METRIC,
        unit_identifier=UNIT_ID,
        cluster_identifier=UNIT_ID,
        is_treated_col='is_treated',
        weight_col='assignments_freq',
        additional_regressors=[],
        start_date=None,
        date_identifier=None,
        lookback=None,
    )._get_results()
    return result.pvalues[1]


# @evaluator
def traditional_cuped(df, metric):
    """Run traditional CUPED and return p-value."""
    
    def _cuped_adjusted_metric(df, metric, metric_pre):
        y = df[metric].values
        x = df[metric_pre].values
        valid_indices = (~np.isnan(y)) & (~np.isnan(x))
        y_valid, x_valid = y[valid_indices], x[valid_indices]
        m = np.cov(y_valid, x_valid)
        theta = m[0, 1] / m[1, 1]
        return (y - (x - np.nanmean(x)) * theta)

    # Perform experiment evaluation and return p-value
    # (Use WLS to be consistent with CausalJet)
    y = _cuped_adjusted_metric(df, metric, f"{metric}_pre")
    x = sm.add_constant(df["is_treated"].astype(float))
    w = df["assignments_freq"]
    model = sm.WLS(endog=y, exog=x, weights=w)
    results = model.fit()
    return results.pvalues["is_treated"]

# @preprocessor
def preproc(df):
    return df

In [27]:
eval = ro.Evaluator(
    df=df,
    metric=METRIC,
    id_col=UNIT_ID,
    evaluators=evaluators,
    sample_min=1000,
    sample_max=21_000,
    num_steps=10,
    mdes=[3],
    num_runs=1,
    baseline_evaluator="welch",
    preprocessors=preprocessors,
    sample_timestamps=False,
    alpha=0.05,
    random_seed=2312,
    testing=True
    )

result = eval.run()
print(result.data.head())
result.plot()

Specified evaluators: ['welch_t_test', 'causal_jet_cuped']
Specified preprocessors: ['preproc']
Generating datasets...


100%|██████████| 10/10 [00:00<00:00, 65.10it/s]


Evaluating experiments...


100%|██████████| 20/20 [00:00<00:00, 245.99it/s]

  preprocessor         evaluator  sample_size  mdes  power
0      preproc  causal_jet_cuped         1000     3    0.0
1      preproc  causal_jet_cuped         3222     3    1.0
2      preproc  causal_jet_cuped         5444     3    1.0
3      preproc  causal_jet_cuped         7667     3    1.0
4      preproc  causal_jet_cuped         9889     3    1.0


alt.VConcatChart(...)

In [12]:
eval = ro.Evaluator(
    df=df,
    preprocessors=preprocessors,
    evaluators=evaluators,
    sample_min=1000,
    sample_max=21_000,
    num_steps=10,
    mdes=[2],
    num_runs=10,
    sample_timestamps=False,
    alpha=0.05,
    metric=METRIC,
    random_seed=2312,
    id_col=UNIT_ID,
    testing=True
    )

result = eval.run()
result

Specified evaluators: ['causal_jet_cuped']


[ExperimentResult(preprocessor='empty_preprocessor', evaluator='causal_jet_cuped', sample_size=1000, mdes=2, power=False),
 ExperimentResult(preprocessor='empty_preprocessor', evaluator='causal_jet_cuped', sample_size=1000, mdes=2, power=False),
 ExperimentResult(preprocessor='empty_preprocessor', evaluator='causal_jet_cuped', sample_size=1000, mdes=2, power=False),
 ExperimentResult(preprocessor='empty_preprocessor', evaluator='causal_jet_cuped', sample_size=1000, mdes=2, power=False),
 ExperimentResult(preprocessor='empty_preprocessor', evaluator='causal_jet_cuped', sample_size=1000, mdes=2, power=False),
 ExperimentResult(preprocessor='empty_preprocessor', evaluator='causal_jet_cuped', sample_size=1000, mdes=2, power=False),
 ExperimentResult(preprocessor='empty_preprocessor', evaluator='causal_jet_cuped', sample_size=1000, mdes=2, power=False),
 ExperimentResult(preprocessor='empty_preprocessor', evaluator='causal_jet_cuped', sample_size=1000, mdes=2, power=False),
 ExperimentResul